<img src="Tut 1.4 Fogler 4.17.jpg" width="900" />

<img src="Tut 1.4.jpg" width="900" />

In [35]:
from numpy import log, array, exp, linspace, asarray
from scipy.integrate import odeint
from scipy.stats import linregress 
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
from scipy.optimize import fsolve

In [37]:
#delta T=0 (isothermal)
#Elementary reaction
#Feed is pure A
W      = 1 #kg
Xpbr   = 0.3
Po     = 20*101.325 # kpa
P      = Po/4 # kpa
FAo    = 1 #kmol basis
FBo    = 0 #kmol - pure A was fed
Fto    = FAo + FBo

**Basis:** $1~\dfrac{kmol}{s}$ Feed

|$Species \qquad $|$Initial \qquad $|$\Delta X \qquad$|$Final \qquad$
|----|-|-|----
|$A$|$1$|$-X$|$(1-X)$
|$B$|$0$|$0.5X$|$0.5X$
|$F_{T}$|$1$|$-0.5X$|$(1 - 0.5X)$

So:
\begin{align}
F_{A_{0}} &= 1 \nonumber \\
F_{B_{0}} &= 0 \nonumber \\
F_{T_{0}} &= 1 \nonumber \\
F_{A} &= 1 - X \nonumber \\
F_{B} &= 0.5X \nonumber \\
F_{T} &= 1 - 0.5X \nonumber \\
Q_{0} &= 1 \nonumber 
\end{align}

The intrinsic rate equation:
\begin{align}
r_{A}^{'} &= -K_{i}^{'}C_{A}^{2} \nonumber \\
&= -K_{i}^{'}\left(\dfrac{F_{A}}{Q}\right)^{2} \nonumber
\end{align}

But:
\begin{align}
Q &= Q_{0} \dfrac{F_{T}}{F_{T_{0}}}\dfrac{T}{T_{0}}\dfrac{P_{0}}{P} \nonumber \\
&= F_{T}\dfrac{P_{0}}{P} \nonumber
\end{align}

Therefore:
\begin{align}
r_{A}^{'} &= -k_{i}^{'}\left(\dfrac{F_{A}P}{F_{T}P_{0}}\right)^{2} \nonumber \\
&= -k_{i}^{'}\left(\dfrac{(1 - X)}{(1 - 0.5X)}\right)^{2}\left(\dfrac{P}{P_{0}} \right)^{2} \nonumber
\end{align}

Isothermal PBR with pressure drop:

Mole balance:
\begin{align}
\dfrac{dF_{A}}{dW} &= r_{A}^{'} \nonumber \\
-F_{A_{0}}\dfrac{dX}{dW} &= r_{A}^{'} \nonumber \\
\dfrac{dX}{dW} &= -r_{A}^{'} \nonumber \\
\dfrac{dX}{dW} &= k_{i}^{'}\left(\dfrac{(1 - X)}{(1 - 0.5X)}\right)^{2}\left(\dfrac{P}{P_{0}} \right)^{2} \nonumber \\
\end{align}

Ergun equation with turbulent flow:
\begin{align}
\dfrac{dP}{dW} &= -K_{Ergun}\left(\dfrac{F_{T}}{F_{T_{0}}}\right)\left(\dfrac{P_{0}}{P}\right) \nonumber \\
\dfrac{dP}{dW} &= -K_{Ergun}(1 - 0.5X)\left(\dfrac{P_{0}}{P}\right) \nonumber \\
\end{align}

In [38]:
kprime = 0.695  #keep changing until final results are P=5atm and X=0.3
K      = 10.431 #keep changing until final results are P=5atm and X=0.3 (this is from ergun eq)

def PBR_K(var,W):
    [X, P] = var
    dXdW = kprime*(((1-X)*P/(1-0.5*X)/Po)**2)
    dPdW = -K*(1-0.5*X)*(Po/P)
    return(dXdW, dPdW)

wspan = linspace(0, W, 100)
ans = odeint(PBR_K,[0,Po],wspan)

X1,P1 = ans.T

print('Conversion=', X1[-1]*100, '%')
print('Final Pressure=', P1[-1], 'atm')

Conversion= 48.65173737178527 %
Final Pressure= 2017.482038120818 atm


**Question a)**

The intrinsic rate equation with no pressure drop:
\begin{align}
r_{A}^{'} &=-K_{i}^{'}\left(\dfrac{(1 - X)}{(1 - 0.5X)}\right)^{2} \nonumber
\end{align}

CSTR design equation:
\begin{align}
F_{A_{0}} - F_{A} + r_{A}^{'}W &= 0 \nonumber \\
F_{A_{0}}X &= k_{i}^{'}C_{A}^{2}.1 \nonumber \\
X &= k_{i}^{'}\left(\dfrac{(1 - X)}{(1 - 0.5X)}\right)^{2} \nonumber
\end{align}

In [46]:
def CSTR(X):
    return(X-kprime*((1-X)/(1-0.5*X))**2)


Conv = fsolve(CSTR,0.2)

print('Conversion= ',Conv*100, '%')

Conversion=  [39.50042007] %


# Question b)

The Ergun equation for turbulent flow:
\begin{equation}
			\frac{dP}{dz} = -\dfrac{1.75G^{2}}{\rho_{\circ}d_{p}}\left(\frac{1 - \epsilon}{\epsilon}\right)\left(\dfrac{P_{\circ}}{P}\frac{F_{T}}{F_{T_{\circ}}}\frac{T}{T_{\circ}}\right)
\end{equation}

For turbulent flow:
\begin{align}
K_{Ergun} = -\dfrac{1.75G^{2}}{\rho_{\circ}d_{p}}\left(\frac{1 - \epsilon}{\epsilon}\right) \nonumber \\
K_{Ergun_{NEW}} = -\dfrac{1.75(G/4)^{2}}{\rho_{\circ}(2d_{p})}\left(\frac{1 - \epsilon}{\epsilon}\right) \nonumber \\
K_{Ergun_{NEW}} = -\dfrac{K_{Ergun}}{(16 \times 2)} \nonumber 
\end{align}

If $\dot{m}$ doubles, then $F_{A_{0}}$ also doubles, consequently:
$$
\begin{align}
r_{A1}^{'} &= -k_{1}^{'}\left(\dfrac{F_{A}}{Q} \right)^{2} \nonumber \\
r_{A2}^{'} &= -k_{1}^{'}\left(\dfrac{2F_{A}}{Q} \right)^{2} \nonumber \\
r_{A2}^{'} &= -4k_{1}^{'}\left(\dfrac{F_{A}}{Q} \right)^{2} \nonumber \\
\therefore k_{2}^{'} &= 4k_{1}^{'} \nonumber
\end{align}
$$

In [47]:
kprime2 = 4*kprime
K2 = K/32

In [48]:
def PBR_b(var,W):
    [X, P] = var
    dXdW = kprime2*(((1-X)*P/(1-0.5*X)/Po)**2) #multiplied by 4 for increase in mass flow rate
    dPdW = -K2*(1-0.5*X)*(Po/P)
    return(dXdW, dPdW)

wspan = linspace(0, W, 100)
sol = odeint(PBR_b,[0,Po],wspan)

X2,P2 = sol.T

print('New Conversion=', X2[-1]*100, '%')
print('Final Pressure=', P2[-1], 'atm')

New Conversion= 86.27213051038598 %
Final Pressure= 2026.2754133072347 atm
